# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 6. Support Vector Machine (SVM)


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-04-21 15:28:43--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-04-21 15:28:43 (51.5 MB/s) - ‘iris.csv’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


Metode Support Vector Machine (SVM) dasar hanya mampu melakukan klasifikasi data yang terdiri dari dua kelas, atau disebut klasifikasi biner. Dataset iris memiliki 3 kelas, sehingga salah satu kelasnya, yaitu Iris-virginica perlu dihapus. Cell berikut menghapus data yang memiliki kelas Iris-virginica

In [4]:
data.drop(data[data['variety']=='Virginica'].index,inplace=True)

SVM juga memiliki keterbatasan, yaitu hanya mampu menerima kelas dalam bentuk numerik. Cell berikut mengubah kelas menjadi bilangan, Iris setosa menjadi -1 dan Iris-versicolor menjadi 1

In [5]:
data['variety']=data['variety'].map({'Setosa':-1,'Versicolor':1})

Tampilkan beberapa data untuk mengecek hasilnya

In [6]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,-1
1,4.9,3.0,1.4,0.2,-1
2,4.7,3.2,1.3,0.2,-1
3,4.6,3.1,1.5,0.2,-1
4,5.0,3.6,1.4,0.2,-1


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [7]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 80 data, dan **data_uji** terdiri dari 20 data

In [8]:
print(data.shape[0])
print(data_uji.shape[0])
print(data_latih.shape[0])

100
20
80


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_latih** dan **label_uji**

In [9]:
label_latih = data_latih.pop('variety')

In [10]:
label_uji = data_uji.pop('variety')

## 3) Proses Training

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [11]:
def hitung_cost_gradient(W,X,Y,regularization):
  jarak = 1 - (Y*np.dot(X,W))
  dw = np.zeros(len(W))
  if max(0,jarak)==0:
    di=W
  else:
    di = W - (regularization * Y*X)
  dw += di
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

In [12]:
from sklearn.utils import shuffle
def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y = shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      delta=hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot=bobot-(learning_rate*delta)
    return bobot

Proses training dilakukan dengan memanggil fungsi **sgd** dengan parameter input berupa data latih dan label latih. Parameter learning rate dan max epoch menggunakan nilai *default* nya

In [13]:
W = sgd(data_latih,label_latih)
print(W)

[-0.14699706 -0.43198222  0.55597346  0.22798929]


## 4) Proses *testing*
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [14]:
def testing(W,data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W,data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi,y_prediksi)
  return prediksi

Lakukan pengujian menggunakan seluruh data uji. Bandingkan hasilnya dengan nilai label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar.

In [15]:
y_prediksi = testing(W,data_uji)
print(sum(y_prediksi==label_uji))

20


## TUGAS
Pada tugas kali ini Anda masih melakukan klasifikasi dataset iris, tetapi menggunakan kelas Iris-versicolor dan Iris-virginica. Apakah hasilnya lebih baik jika dibandingkan dengan Iris-setosa dan Iris-versicolor?

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
data = pd.read_csv('iris.csv')
data = data[data['variety'] != 'Setosa']
data['variety'] = data['variety'].map({'Versicolor':-1,'Virginica':1})

In [39]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('variety', axis=1), data['variety'], test_size=0.2, random_state=42)

def hitung_cost_gradient(W,X,Y,regularization):
    jarak = 1 - (Y*np.dot(X,W))
    dw = np.zeros(len(W))
    if max(0,jarak)==0:
        di=W
    else:
        di = W - (regularization * Y*X)
    dw += di
    return dw

def sgd(X_train, y_train, learning_rate=0.000001, max_epoch=1000, regularization=10000):
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()
    W = np.zeros(X_train.shape[1])
    for epoch in range(max_epoch):
        X_train, y_train = shuffle(X_train, y_train, random_state=101)
        for i in range(X_train.shape[0]):
            delta = hitung_cost_gradient(W, X_train[i], y_train[i], regularization)
            W = W - (learning_rate * delta)
    return W

In [40]:
W = sgd(X_train, y_train)

def testing(W,X):
    y_pred = np.array([])
    for i in range(X.shape[0]):
        y_prediksi = np.sign(np.dot(W,X.to_numpy()[i]))
        y_pred = np.append(y_pred, y_prediksi)
    return y_pred

In [41]:
y_pred = testing(W, X_test)
pred = np.mean(y_pred == y_test)
print(pred)

0.85


Hasilnya tidak lebih baik karena hanya mampu mengenali 17 diantara 20 data uji.

Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi **sgd**. Apakah hasilnya lebih baik? Mengapa?

In [43]:
W = sgd(X_train, y_train, learning_rate=0.000001, max_epoch=5000, regularization=10000)
print(W)

y_pred = testing(W, X_test)
print(sum(y_pred == y_test))


[ -5.12571519 -10.02455856   9.74720645   8.35358996]
17


Terakhir, lakukanlah seleksi fitur sederahana dengan mengambil hanya fitur sepal length dan sepal width saja. Apakah hasilnya lebih baik?

In [45]:
data = pd.read_csv('iris.csv')
data_sepal = data[['sepal.length', 'sepal.width', 'variety']]
data_sepal.drop(data_sepal[data_sepal['variety'] == 'Virginica'].index, inplace=True)
data_sepal['variety'] = data_sepal['variety'].map({'Setosa': -1, 'Versicolor': 1})
data_latih_sepal, data_uji_sepal = train_test_split(data_sepal, test_size=0.2)
label_latih_sepal = data_latih_sepal.pop('variety')
label_uji_sepal = data_uji_sepal.pop('variety')
W_sepal = sgd(data_latih_sepal, label_latih_sepal, learning_rate=0.000001, max_epoch=1000, regularization=10000)
print(W_sepal)
y_prediksi_sepal = testing(W_sepal, data_uji_sepal)
print(sum(y_prediksi_sepal == label_uji_sepal))

<ipython-input-45-b4d21f010901>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sepal.drop(data_sepal[data_sepal['variety'] == 'Virginica'].index, inplace=True)
<ipython-input-45-b4d21f010901>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sepal['variety'] = data_sepal['variety'].map({'Setosa': -1, 'Versicolor': 1})


[ 3.62625821 -6.35872745]
20
